In [1]:
import torch
import sys
sys.path.append('..')
from src.utils.model_utils import load_model
from tokenizers import Tokenizer
from src.data.text_dataset import TextDataset
from src.data.hf_dataset_processor import HFDatasetProcessor
from torch.utils.data import DataLoader
import torch.nn.functional as F

/Users/aszarata/informatyka/9s/lingwistyka/lab02/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = Tokenizer.from_file("../models/model-bpe/tokenizer.json")
model = load_model("../models/model-bpe/checkpoint_6")

In [3]:
sum([p.numel() for p in model.parameters()])


109158736

In [4]:
dataset = TextDataset(
    data_dir="../data/base/test",
    tokenizer=tokenizer,
    seq_len=128
)

test_loader = DataLoader(
    dataset=dataset, 
    batch_size=1,
    shuffle=True
)

In [5]:
dataset2 = HFDatasetProcessor(
    "../data/plwiki",
    tokenizer=tokenizer,
    max_seq_len=128
).get_data()
dataloader = DataLoader(
    dataset=dataset2, 
    batch_size=4,
    shuffle=True
)

Tokenizing dataset:   3%|▎         | 46000/1562327 [00:15<08:25, 3000.26 examples/s]


KeyboardInterrupt: 

In [6]:
x, y = next(iter(test_loader))
x2, y2 = next(iter(dataloader))

NameError: name 'dataloader' is not defined

In [18]:
print(x2.shape, y2.shape)

torch.Size([4, 128]) torch.Size([4, 128])


In [23]:
print(x2[3], y2[3])

tensor([13325,  8951,  1267,   678,    17,  2094,    85, 17726,    12, 18095,
        10623,  1241, 19251,   446, 45940, 25497,    15, 27159,   155, 39674,
           16, 44778,  2142,  2618,  1958,    15,   181, 20571,  3285, 35473,
          498, 45940, 18599,    15,  4673,   859,    67,  1572,  7381, 26283,
          310,   453,   504,   222,    17,   346,   454,  1714,  8211,   196,
         5783,  2659,    17,  8470, 30764,  5117,    17,   127,   127,    42,
           69, 13257,    65,   127,   127,  8043,   206,  3068, 14185,  1241,
          129,   127, 34862,   287,   446, 13325,  8951,  3872,   155, 39674,
           16, 44778,  2142,  2618, 45940, 25497,    17,   462,  6619,  2198,
          226,   159,  7543, 21343,  1212,   824, 23710,    17,   462, 13006,
         1630,    90,  7083,  9093,  1694,  5146,   159,  2448,   163,   492,
          170,   442, 23485,  2106,    15,   470,   181,  5888,   159,  2408,
         2171,   330,  1715,   170,   181, 12018,   159,   310])

In [9]:
logits = model(x)

log_probs = F.log_softmax(logits, dim=-1)

In [10]:
print(tokenizer.decode(x[0].tolist()))

 którzy na komorne wprawdzie nie mają, ale urządzać u siebie hałasy i pijatyki co tydzień to mogą, ujmuje się za Ignacym, starym leniem i niedołęgą, przez którego marnieje ogród, nie da słowa powiedzieć żadnej służącej. Co to wszystko jest? Poczucie sprawiedliwości, równość, uświadomienie społeczne? Pani Cecylii wiadome były te rzeczy nie od dzisiaj. Sprawiedliwość, dobrze. Ale niechże idzie ze swoimi pretensjami tam, gdzie naprawdę jest jakiś wyzysk i krzywda, nie do niej, którą właśnie wszyscy wyzyskują, której ta kamienica nic nie przynosi prócz kłopotów. „Ja ci wcale nie powiadam — mówiła do Elżbiety


In [11]:
print(tokenizer.decode(y[0].tolist()))

 na komorne wprawdzie nie mają, ale urządzać u siebie hałasy i pijatyki co tydzień to mogą, ujmuje się za Ignacym, starym leniem i niedołęgą, przez którego marnieje ogród, nie da słowa powiedzieć żadnej służącej. Co to wszystko jest? Poczucie sprawiedliwości, równość, uświadomienie społeczne? Pani Cecylii wiadome były te rzeczy nie od dzisiaj. Sprawiedliwość, dobrze. Ale niechże idzie ze swoimi pretensjami tam, gdzie naprawdę jest jakiś wyzysk i krzywda, nie do niej, którą właśnie wszyscy wyzyskują, której ta kamienica nic nie przynosi prócz kłopotów. „Ja ci wcale nie powiadam — mówiła do Elżbiety —


In [12]:
print(log_probs.shape)
predicted_tokens = log_probs.argmax(dim=-1)

decoded_text = tokenizer.decode(predicted_tokens[0].tolist())
print(decoded_text)

torch.Size([1, 128, 50000])



In [13]:
mask = (y != tokenizer.token_to_id("<pad>"))
target_log_probs = log_probs.gather(
    dim=-1,
    index=y.unsqueeze(-1)

).squeeze(-1)
masked_log_probs = target_log_probs * mask

negative_log_likelihood = -masked_log_probs.sum()
meana = negative_log_likelihood.item() / mask.sum()
torch.exp(meana)

tensor(nan)

In [14]:
torch.exp(target_log_probs)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan]], grad_fn=<ExpBackward0>)

In [15]:
probs = F.softmax(logits, dim=-1)
print(torch.max(probs[0][0]))
print(torch.argmax(probs[0][0]))
print(probs[0][0][2810])
print(y[0][0])
print(type(y[0][0].item()))
print(tokenizer.decode([y[0][0].item()]))

tensor(nan, grad_fn=<MaxBackward1>)
tensor(0)
tensor(nan, grad_fn=<SelectBackward0>)
tensor(181)
<class 'int'>
 na
